In [96]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.1.2-bin-hadoop2.7'

In [97]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("how to read csv file") \
    .getOrCreate()
path = "C:\\Users\\nambi\\OneDrive\\Desktop\\Internship\\transactions.json"


In [98]:
from pyspark.sql import types as T
import pyspark.sql.functions as F


def flatten(df):
    complex_fields = dict([
        (field.name, field.dataType) 
        for field in df.schema.fields 
        if isinstance(field.dataType, T.ArrayType) or isinstance(field.dataType, T.StructType)
    ])
    
    qualify = list(complex_fields.keys())[0] + "_"

    while len(complex_fields) != 0:
        col_name = list(complex_fields.keys())[0]
        
        if isinstance(complex_fields[col_name], T.StructType):
            expanded = [F.col(col_name + '.' + k).alias(col_name + '_' + k) 
                        for k in [ n.name for n in  complex_fields[col_name]]
                       ]
            
            df = df.select("*", *expanded).drop(col_name)
    
        elif isinstance(complex_fields[col_name], T.ArrayType): 
            df = df.withColumn(col_name, F.explode(col_name))
    
      
        complex_fields = dict([
            (field.name, field.dataType)
            for field in df.schema.fields
            if isinstance(field.dataType, T.ArrayType) or isinstance(field.dataType, T.StructType)
        ])
        
        
    for df_col_name in df.columns:
        df = df.withColumnRenamed(df_col_name, df_col_name.replace(qualify, ""))

    return df

df = spark.read.json(path)
#peopleDF.show()
df_trans=flatten(df)
df_trans.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- date_of_purchase: string (nullable = true)
 |-- price: long (nullable = true)
 |-- product_id: string (nullable = true)



In [99]:
#df_trans['purchase_count_per_product_id']=df_trans.withColumn()
#df_trans = 
df_purchase = df_trans.groupBy("product_id") \
    .agg(count("product_id").alias("purchase_count_per_product_id"))
df_purchase.show(50)

+----------+-----------------------------+
|product_id|purchase_count_per_product_id|
+----------+-----------------------------+
|       P13|                            2|
|       P23|                            2|
|        P2|                            2|
|       P15|                            2|
|       P11|                            2|
|        P7|                            2|
|       P21|                            2|
|        P8|                            2|
|       P14|                            2|
|        P3|                            2|
|       P10|                            2|
|       P16|                            2|
|        P4|                            2|
|        P1|                            1|
|        P5|                            2|
|       P17|                            2|
|       P12|                            2|
|       P22|                            2|
|       P19|                            2|
|       P24|                            1|
|       P18

In [100]:
df_customer = spark.read.csv('C:\\Users\\nambi\\OneDrive\\Desktop\\Internship\\customers.csv',header=True)
df_customer.show()

+-----------+-------------+
|customer_id|loyalty_score|
+-----------+-------------+
|         C1|         0.67|
|         C2|         0.78|
|         C3|         0.81|
|         C4|         0.46|
|         C5|         0.98|
|         C6|         0.78|
|         C7|         0.67|
|         C8|         0.99|
|         C9|         0.88|
|        C10|         0.23|
+-----------+-------------+



In [101]:
df_prod=spark.read.csv('C:\\Users\\nambi\\OneDrive\\Desktop\\Internship\\Products.csv',header=True)
df_prod.show()

+----------+-------------------+----------------+
|product_id|product_description|product_category|
+----------+-------------------+----------------+
|        P1|              Paste|         Grocery|
|        P2|              Apple|          Fruits|
|        P3|               Kiwi|          Fruits|
|        P4|      Hide and seek|        Biscuits|
|        P5|              Brush|         Grocery|
|        P6|                Dal|          Grains|
|        P7|              Wheat|          Grains|
|        P8|          Bournvita|         Grocery|
|        P9|               Mask|         Grocery|
|       P10|        Nail cutter|         Grocery|
|       P11|              Boost|         Grocery|
|       P12|            Cabbage|          Vegies|
|       P13|             Carrot|          Vegies|
|       P14|             Banana|          Fruits|
|       P15|               Eggs|         Grocery|
|       P16|            Chicken|            Meat|
|       P17|             Tomato|          Vegies|


In [102]:
df_final=df_trans.join(df_customer,["customer_id"]) \
     .join(df_prod,["product_id"]) \
     .join(df_purchase,["product_id"])

In [103]:
df_final2 = df_final[["customer_id","loyalty_score","product_id","product_category","purchase_count_per_product_id"]]
df_final2.show()

+-----------+-------------+----------+----------------+-----------------------------+
|customer_id|loyalty_score|product_id|product_category|purchase_count_per_product_id|
+-----------+-------------+----------+----------------+-----------------------------+
|         C1|         0.67|        P1|         Grocery|                            1|
|         C1|         0.67|        P2|          Fruits|                            2|
|         C2|         0.78|        P2|          Fruits|                            2|
|         C2|         0.78|        P3|          Fruits|                            2|
|         C3|         0.81|        P3|          Fruits|                            2|
|         C3|         0.81|        P4|        Biscuits|                            2|
|         C4|         0.46|        P4|        Biscuits|                            2|
|         C4|         0.46|        P5|         Grocery|                            2|
|         C5|         0.98|        P5|         Grocery